In [ ]:
import pandas as pd
import os
import s3fs
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import numpy as np
import json


# Create filesystem object
S3_ENDPOINT_URL = "https://" + os.environ["AWS_S3_ENDPOINT"]
fs = s3fs.S3FileSystem(client_kwargs={"endpoint_url": S3_ENDPOINT_URL})

BUCKET = ""  # bucket path
FILE_KEY_S3 = "df_all_clusters_final.csv"
FILE_PATH_S3 = BUCKET + FILE_KEY_S3

with fs.open(FILE_PATH_S3, mode="rb") as file_in:
    dataset = pd.read_csv(file_in, sep=",")

In [ ]:
with fs.open(BUCKET + "df_Données_Reabos_odd_final.csv", mode="rb") as file_in:
    bigdata = pd.read_csv(file_in, sep=",")
print(bigdata["ID_ABONNE"].nunique())

In [ ]:
print(dataset["ID_ABONNE"].nunique())
print(dataset.columns)
dataset

In [ ]:
def date_bascule(df, cluster):
    data = df.copy()
    data = data.loc[data["Cluster_8"] == cluster]
    data["DATE_PRISE_EFFET"] = pd.to_datetime(data["DATE_PRISE_EFFET"])
    data["ANNEE_MOIS"] = data["DATE_PRISE_EFFET"].dt.strftime("%Y-%m")
    result = data.groupby("ANNEE_MOIS")["ID_ABONNE"].nunique().reset_index()
    result.rename(columns={"ID_ABONNE": "NOMBRE_ABONNEMENTS"}, inplace=True)

    def ecart_type_avant_après(dataframe, line):
        result_before = dataframe.iloc[line:]
        result_after = dataframe.iloc[:line]
        std_before = result_before["NOMBRE_ABONNEMENTS"].std()
        mean_before = result_before["NOMBRE_ABONNEMENTS"].mean()
        std_after = result_after["NOMBRE_ABONNEMENTS"].std()
        mean_after = result_after["NOMBRE_ABONNEMENTS"].mean()
        return (std_before, std_after, mean_before, mean_after)

    min_std = float("inf")
    date = None
    min_std_before = None
    min_std_after = None
    the_mean_before = None
    the_mean_after = None
    for line in range(1, result.shape[0]):
        std_before, std_after, mean_before, mean_after = ecart_type_avant_après(
            result, line
        )
        std_line = std_before + std_after
        if min_std > std_line:
            min_std = std_line
            min_std_before = std_before
            min_std_after = std_after
            the_mean_before = mean_before
            the_mean_after = mean_after
            date = line

    ratio = the_mean_before / the_mean_after
    if ratio > 19 / 20 and ratio < 20 / 19:
        date = None
        min_std_before = None
        min_std_after = None
        the_mean_before = None
        the_mean_after = None

    if date != None:
        mois_bascule = result.loc[date, "ANNEE_MOIS"]
        print("Date = ", mois_bascule)
    else:
        print("Date = ", date)
    print("Std error before = ", min_std_before)
    print("Std error after = ", min_std_after)
    print("Mean before = ", the_mean_before)
    print("Mean after = ", the_mean_after)
    print("Ecart des moyennes = ")

    return (result, mois_bascule)


liste_bascule = []
for i in range(0, 8, 1):
    print(i)
    liste_bascule += [date_bascule(dataset, i)[1]]
print(liste_bascule)

In [ ]:
def find_periods(df, cluster):

    data = df.copy()
    data = data.loc[data["Cluster_8"] == cluster]
    data["DATE_PRISE_EFFET"] = pd.to_datetime(data["DATE_PRISE_EFFET"])
    result = (
        data.groupby(["DATE_PRISE_EFFET", "TYPE_PROMON"])["ID_ABONNE"]
        .nunique()
        .reset_index()
    )
    result.rename(columns={"ID_ABONNE": "NOMBRE_ABONNEMENTS"}, inplace=True)

    # Calculer la moyenne mobile sur une fenêtre glissante pour chaque TYPE_PROMON
    window_size = 7  # Taille de la fenêtre glissante pour la moyenne mobile
    df["rolling_mean"] = df.groupby("TYPE_PROMON")["NOMBRE_ABONNEMENTS"].transform(
        lambda x: x.rolling(window_size).mean()
    )

    # Identifier les périodes où une promotion est significativement plus élevée que les autres
    # Par exemple, vous pouvez définir un seuil pour déterminer ce qui constitue une augmentation significative
    seuil = 1.5  # Changer cela selon vos besoins
    promotions_significatives = df[
        df["NOMBRE_ABONNEMENTS"] > seuil * df["rolling_mean"]
    ]

    # Afficher les dates qui délimitent ces périodes
    dates_limites = promotions_significatives["DATE_PRISE_EFFET"].unique()

    print(
        "Les dates limites des périodes où une promotion est significativement plus utilisée que les autres sont :",
        dates_limites,
    )

    return result


find_periods(dataset, 0)

In [ ]:
def create_dataset_all_dates(df, cluster):

    data = df.copy()
    data = data.loc[data["Cluster_8"] == cluster]
    """
    data.rename(columns={'DATE_PRISE_EFFET' : 'NOUVEAU_NOM'}, inplace = True)
    data.rename(columns={'DATE_ACTE_REEL' : 'DATE_PRISE_EFFET'}, inplace = True)
    """
    data["DATE_PRISE_EFFET"] = pd.to_datetime(data["DATE_PRISE_EFFET"])
    result = (
        data.groupby(["DATE_PRISE_EFFET", "TYPE_PROMON"])["ID_ABONNE"]
        .nunique()
        .reset_index()
    )
    result.rename(columns={"ID_ABONNE": "NOMBRE_ABONNEMENTS"}, inplace=True)

    # Création d'un DataFrame contenant toutes les dates et tous les types promon possibles
    min_date = result["DATE_PRISE_EFFET"].min()
    max_date = result["DATE_PRISE_EFFET"].max()
    all_dates = pd.date_range(start=min_date, end=max_date, freq="D")
    all_promotions = result["TYPE_PROMON"].unique()
    all_possible_combinations = pd.MultiIndex.from_product(
        [all_dates, all_promotions], names=["DATE_PRISE_EFFET", "TYPE_PROMON"]
    )
    df_all_combinations = pd.DataFrame(index=all_possible_combinations).reset_index()

    # Fusion avec votre DataFrame initial pour remplir les valeurs manquantes
    merged_df = pd.merge(
        df_all_combinations, result, on=["DATE_PRISE_EFFET", "TYPE_PROMON"], how="left"
    )

    # Remplacement des valeurs manquantes de NOMBRE_ABONNEMENTS par 0
    merged_df["NOMBRE_ABONNEMENTS"] = merged_df["NOMBRE_ABONNEMENTS"].fillna(0)

    return merged_df


print(create_dataset_all_dates(dataset, 0))
create_dataset_all_dates(dataset, 0)

In [ ]:
def find_periods(data, cluster):
    df = create_dataset_all_dates(data, cluster)

    # Étape 1 : Identifier les périodes où le nombre d'abonnements est significativement plus élevé que la moyenne ou la médiane
    periodes_significatives = {}
    for promotion, series in df.groupby("TYPE_PROMON"):
        # Calculer la moyenne ou la médiane de la série temporelle
        mean = series["NOMBRE_ABONNEMENTS"].mean()
        # Remplacer mean par median pour utiliser la médiane
        std = series["NOMBRE_ABONNEMENTS"].std()
        seuil = (
            mean + 2 * std
        )  # Choisir un seuil approprié en fonction de votre analyse
        # Identifier les périodes où le nombre d'abonnements dépasse le seuil
        periodes_significatives[promotion] = series[
            series["NOMBRE_ABONNEMENTS"] > seuil
        ]

    # Étape 2 : Fusionner les périodes consécutives avec le même TYPE_PROMON
    merged_periodes = {}
    for promotion, periodes in periodes_significatives.items():
        current_start = None
        current_end = None
        for index, row in periodes.iterrows():
            if current_start is None:
                current_start = row["DATE_PRISE_EFFET"]
                current_end = row["DATE_PRISE_EFFET"]
            elif row["DATE_PRISE_EFFET"] == current_end + pd.Timedelta(days=1):
                current_end = row["DATE_PRISE_EFFET"]
            else:
                if promotion not in merged_periodes:
                    merged_periodes[promotion] = []
                merged_periodes[promotion].append((current_start, current_end))
                current_start = row["DATE_PRISE_EFFET"]
                current_end = row["DATE_PRISE_EFFET"]
        if current_start is not None:
            if promotion not in merged_periodes:
                merged_periodes[promotion] = []
            merged_periodes[promotion].append((current_start, current_end))

    # Étape 3 : Fusionner les périodes qui se chevauchent avec des TYPE_PROMON différents
    final_periodes = []
    for promotion, periodes in merged_periodes.items():
        for periode in periodes:
            if not final_periodes:
                final_periodes.append((promotion, periode[0], periode[1]))
            else:
                last_promotion, last_start, last_end = final_periodes[-1]
                if last_end >= periode[0] and last_promotion != promotion:
                    final_periodes[-1] = (
                        last_promotion + " / " + promotion,
                        min(last_start, periode[0]),
                        max(last_end, periode[1]),
                    )
                else:
                    final_periodes.append((promotion, periode[0], periode[1]))

    # Étape 4 : Affichez les périodes résultantes avec les TYPE_PROMON et les nombres d'abonnements associés
    for promotion, debut_periode, fin_periode in final_periodes:
        periode_df = df[
            (df["DATE_PRISE_EFFET"] >= debut_periode)
            & (df["DATE_PRISE_EFFET"] <= fin_periode)
            & (df["TYPE_PROMON"].str.contains(promotion))
        ]
        total_abonnements = periode_df["NOMBRE_ABONNEMENTS"].sum()
        print(
            f"Période significative pour {promotion} du {debut_periode} au {fin_periode}, Nombre total d'abonnements : {total_abonnements}"
        )
    return


for i in range(0, 8, 1):
    find_periods(dataset, i)

In [ ]:
def find_percent(df):
    """
    Détermine le pourcentage d'abonnés dans le cluster
    """
    data = df.copy()
    total_abonnes = data["ID_ABONNE"].nunique()

    pourcentages_clusters = {}
    for cluster in range(0, 8, 1):
        abonnes_cluster = data[data["Cluster_8"] == cluster]["ID_ABONNE"].nunique()
        pourcentage_cluster = (abonnes_cluster / total_abonnes) * 100
        pourcentages_clusters[cluster] = pourcentage_cluster

    # Afficher les pourcentages associés à chaque cluster
    for cluster, pourcentage in pourcentages_clusters.items():
        print(f"Cluster {cluster}: {pourcentage:.2f}%")
    return pourcentages_clusters


find_percent(dataset)

In [ ]:
def create_periods(date_debut, nb_periods, periode_length_months):
    """Crée les périodes en fonction de la date de début, du nombre de périodes et de la longueur de chaque période en mois."""
    periods = []
    for i in range(nb_periods):
        debut_periode = date_debut + timedelta(days=periode_length_months * 30 * i)
        fin_periode = debut_periode + timedelta(days=periode_length_months * 30)
        periods += [(debut_periode, fin_periode)]
    return periods


def calculate_abonnements_moyens(datata, periods):
    """Calcule le nombre d'abonnements moyens par jour de disponibilité pour chaque cluster sur chaque période."""
    df = datata.copy()
    df["DATE_PRISE_EFFET"] = pd.to_datetime(df["DATE_PRISE_EFFET"])
    nombre_abonnements_moyens = {}
    for cluster, group in df.groupby("Cluster_8"):
        cluster_abonnements_moyens = {}
        for i, (debut_periode, fin_periode) in enumerate(periods, start=1):
            print("Cluster ", cluster, "période ", i, (debut_periode, fin_periode))
            periode_data = group[
                (group["DATE_PRISE_EFFET"] >= debut_periode)
                & (group["DATE_PRISE_EFFET"] < fin_periode)
            ]
            # abonnements_periode = periode_data.groupby('ID_ABONNE')['DATE_PRISE_EFFET'].nunique().count()
            abonnements_periode = periode_data.shape[0]
            print("abonnements_periode :", abonnements_periode)
            if i < 6:
                jours_disponibilite = 6 * periode_data["ID_ABONNE"].nunique()
            elif i == 6:
                jours_disponibilite = 4 * periode_data["ID_ABONNE"].nunique()

            print("jours_disponibilite : ", jours_disponibilite)
            if jours_disponibilite > 0:
                abonnements_moyens_periode = abonnements_periode / jours_disponibilite
            else:
                abonnements_moyens_periode = 0
            cluster_abonnements_moyens[f"Période {i}"] = abonnements_moyens_periode
        nombre_abonnements_moyens[cluster] = cluster_abonnements_moyens
    return nombre_abonnements_moyens


# Définir la date de début et le nombre de périodes
date_debut = datetime.strptime("2021-01-01", "%Y-%m-%d")
nb_periods = 6
periode_length_months = 6

# Créer les périodes
periods = create_periods(date_debut, nb_periods, periode_length_months)
print(periods)

calculate_abonnements_moyens(dataset, periods)

In [ ]:
def find_mean_with_promo(df):
    data = df.copy()
    # Grouper les données par CLUSTER_8 et TYPE_PROMON
    grouped_data = data.groupby(["Cluster_8", "TYPE_PROMON"])

    # Initialiser des dictionnaires pour stocker les résultats
    nombre_interventions = {}
    nombre_dates_differentes = {}
    nombre_reabo_par_jour_dispo = {}

    # Boucler à travers chaque groupe
    for (cluster, type_promo), group in grouped_data:
        # Nombre de fois que TYPE_PROMON i intervient dans le cluster_8 j
        nombre_intervention = len(group)
        # Nombre de DATE_PRISE_EFFET différentes auxquelles TYPE_PROMON i intervient dans le cluster_8 j
        nombre_dates_diff = group["DATE_PRISE_EFFET"].nunique()

        # Stocker les résultats dans les dictionnaires
        if cluster not in nombre_interventions:
            nombre_interventions[cluster] = {}
            nombre_dates_differentes[cluster] = {}
            nombre_reabo_par_jour_dispo[cluster] = {}
        nombre_interventions[cluster][type_promo] = nombre_intervention
        nombre_dates_differentes[cluster][type_promo] = nombre_dates_diff
        nombre_reabo_par_jour_dispo[cluster][type_promo] = (
            nombre_intervention / nombre_dates_diff
        )

    # Afficher les résultats
    for cluster, interventions in nombre_interventions.items():
        print(f"Cluster {cluster}:")
        for type_promo, nb_interventions in interventions.items():
            nb_dates_diff = nombre_dates_differentes[cluster][type_promo]
            print(f"  - Type promotion {type_promo}:")
            # print(f"      Nombre d'interventions : {nb_interventions}")
            # print(f"      Nombre de dates différentes : {nb_dates_diff}")
            print(
                f"      Nombre de réabonnements moyens par jour de disponibilité : {nb_interventions/nb_dates_diff}"
            )
    return nombre_reabo_par_jour_dispo


find_mean_with_promo(dataset)

In [ ]:
def find_mean_with_promo_periods(df, periods):
    print(periods)
    data = df.copy()
    data["DATE_PRISE_EFFET"] = pd.to_datetime(data["DATE_PRISE_EFFET"])

    # Grouper les données par CLUSTER_8, TYPE_PROMON et Période
    nombre_reabo_par_jour_dispo = {}
    for (cluster, type_promo), group in data.groupby(["Cluster_8", "TYPE_PROMON"]):
        cluster_abonnements_moyens = {}
        for i, (debut_periode, fin_periode) in enumerate(periods, start=1):
            periode_data = group[
                (group["DATE_PRISE_EFFET"] >= debut_periode)
                & (group["DATE_PRISE_EFFET"] < fin_periode)
            ]
            abonnements_periode = len(periode_data)
            dates_diff = periode_data["DATE_PRISE_EFFET"].nunique()
            if dates_diff > 0:
                abonnements_moyens_periode = abonnements_periode / dates_diff
            else:
                abonnements_moyens_periode = 0
            cluster_abonnements_moyens[f"Période {i}"] = abonnements_moyens_periode
        if cluster not in nombre_reabo_par_jour_dispo:
            nombre_reabo_par_jour_dispo[cluster] = {}
        nombre_reabo_par_jour_dispo[cluster][type_promo] = cluster_abonnements_moyens

    # Afficher les résultats
    for cluster, resultats_periode in nombre_reabo_par_jour_dispo.items():
        print(f"Cluster {cluster}:")
        for type_promo, resultats_periode_type in resultats_periode.items():
            print(f"  - Type promotion {type_promo}:")
            for periode, moyenne in resultats_periode_type.items():
                print(
                    f"      Période {periode}: Nombre de réabonnements moyens par jour de disponibilité : {moyenne}"
                )
    return nombre_reabo_par_jour_dispo


# Appeler la fonction pour calculer les abonnements moyens
find_mean_with_promo_periods(dataset, periods)

On crée le df qui regroupe toutes les informations

In [ ]:
dataset_cluster_explanations = pd.DataFrame({"Cluster": list(range(8))})

dataset_cluster_explanations["Pourcentage de la population"] = (
    dataset_cluster_explanations["Cluster"].map(find_percent(dataset))
)
dataset_cluster_explanations["Dico"] = dataset_cluster_explanations["Cluster"].map(
    find_mean_with_promo_periods(dataset, periods)
)
print(dataset["TYPE_PROMON"].unique())
for i in range(1, 7, 1):
    for promo in dataset["TYPE_PROMON"].unique():
        dataset_cluster_explanations[
            f"Facteur d'abonnement pour {promo} sur la période {i}"
        ] = dataset_cluster_explanations["Dico"].apply(
            lambda x: x[promo][f"Période {i}"]
        )

for promo in dataset["TYPE_PROMON"].unique():
    for i in range(1, 7, 1):
        if not dataset_cluster_explanations[
            f"Facteur d'abonnement pour Semaine genéreuse sur la période {i}"
        ].empty:
            dataset_cluster_explanations[
                f"Facteur d'abonnement pour {promo} sur la période {i}"
            ] = (
                dataset_cluster_explanations[
                    f"Facteur d'abonnement pour {promo} sur la période {i}"
                ]
                / dataset_cluster_explanations[
                    f"Facteur d'abonnement pour Semaine genéreuse sur la période {i}"
                ]
            )
        else:
            dataset_cluster_explanations[
                f"Facteur d'abonnement pour {promo} sur la période {i}"
            ] = float("inf")

dataset_cluster_explanations.drop(columns=["Dico"], inplace=True)

dataset_cluster_explanations["Periods"] = dataset_cluster_explanations["Cluster"].map(
    calculate_abonnements_moyens(dataset, periods)
)
print(dataset_cluster_explanations)
for i in range(1, 7, 1):
    dataset_cluster_explanations[f"Taux d'abonnement sur la période {i}"] = (
        dataset_cluster_explanations["Periods"].apply(lambda x: x[f"Période {i}"])
    )
dataset_cluster_explanations.drop(columns=["Periods"], inplace=True)
dataset_cluster_explanations = dataset_cluster_explanations.fillna(0)
dataset_cluster_explanations = dataset_cluster_explanations.round(1)

# Enregistrer la figure dans un fichier
nom_fichier_temporaire = "clusters_explanation.csv"
dataset_cluster_explanations.to_csv(nom_fichier_temporaire, index=False)
# Enregistrer le fichier temporaire sur S3
with fs.open(BUCKET + nom_fichier_temporaire, mode="wb") as file_out:
    with open(nom_fichier_temporaire, "rb") as temp_file:
        file_out.write(temp_file.read())
# Supprimer le fichier temporaire
os.remove(nom_fichier_temporaire)

dataset_cluster_explanations

In [ ]:
dataset_cluster_explanations = pd.DataFrame({"Cluster": list(range(8))})


dataset_cluster_explanations["Pourcentage de la population"] = (
    dataset_cluster_explanations["Cluster"].map(find_percent(dataset))
)
dataset_cluster_explanations["Dico"] = dataset_cluster_explanations["Cluster"].map(
    find_mean_with_promo_periods(dataset, periods)
)
print(dataset_cluster_explanations["Dico"])
print(dataset["TYPE_PROMON"].unique())
for i in range(1, 7, 1):
    for promo in ["Semaine genéreuse", "ODD 15 jours TC", "ODD 30 jours TC"]:
        dataset_cluster_explanations[
            f"Facteur d'abonnement pour {promo} sur la période {i}"
        ] = dataset_cluster_explanations["Dico"].apply(
            lambda x: x[promo][f"Période {i}"]
        )

for promo in ["Semaine genéreuse", "ODD 15 jours TC", "ODD 30 jours TC"]:
    for i in range(1, 7, 1):
        if not dataset_cluster_explanations[
            f"Facteur d'abonnement pour Semaine genéreuse sur la période {i}"
        ].empty:
            dataset_cluster_explanations[
                f"Facteur d'abonnement pour {promo} sur la période {i}"
            ] = (
                dataset_cluster_explanations[
                    f"Facteur d'abonnement pour {promo} sur la période {i}"
                ]
                / dataset_cluster_explanations[
                    f"Facteur d'abonnement pour Semaine genéreuse sur la période {i}"
                ]
            )
        else:
            dataset_cluster_explanations[
                f"Facteur d'abonnement pour {promo} sur la période {i}"
            ] = float("inf")

dataset_cluster_explanations.drop(columns=["Dico"], inplace=True)

dataset_cluster_explanations["Periods"] = dataset_cluster_explanations["Cluster"].map(
    calculate_abonnements_moyens(dataset, periods)
)
print(dataset_cluster_explanations)
for i in range(1, 7, 1):
    dataset_cluster_explanations[f"Taux d'abonnement sur la période {i}"] = (
        dataset_cluster_explanations["Periods"].apply(lambda x: x[f"Période {i}"])
    )
dataset_cluster_explanations.drop(columns=["Periods"], inplace=True)
dataset_cluster_explanations = dataset_cluster_explanations.fillna(0)
dataset_cluster_explanations = dataset_cluster_explanations.round(1)

# Enregistrer la figure dans un fichier
nom_fichier_temporaire = "clusters_explanation_shortened_2.csv"
dataset_cluster_explanations.to_csv(nom_fichier_temporaire, index=False)
# Enregistrer le fichier temporaire sur S3
with fs.open(BUCKET + nom_fichier_temporaire, mode="wb") as file_out:
    with open(nom_fichier_temporaire, "rb") as temp_file:
        file_out.write(temp_file.read())
# Supprimer le fichier temporaire
os.remove(nom_fichier_temporaire)

dataset_cluster_explanations

In [ ]:
dataset_cluster_explanations = pd.DataFrame({"Cluster": list(range(8))})
print(dataset_cluster_explanations)

dataset_cluster_explanations["Pourcentage de la population"] = (
    dataset_cluster_explanations["Cluster"].map(find_percent(dataset))
)
dataset_cluster_explanations["Dico"] = dataset_cluster_explanations["Cluster"].map(
    find_mean_with_promo_periods(dataset, periods)
)
print(dataset["TYPE_PROMON"].unique())
list_promo = ["Semaine genéreuse", "ODD 15 jours TC", "ODD 30 jours TC"]
for i in range(1, 7, 1):
    for promo in list_promo:
        dataset_cluster_explanations[
            f"Facteur d'abonnement pour {promo} sur la période {i}"
        ] = dataset_cluster_explanations["Dico"].apply(
            lambda x: x[promo][f"Période {i}"]
        )

for promo in list_promo:
    for i in range(1, 7, 1):
        if not dataset_cluster_explanations[
            f"Facteur d'abonnement pour Semaine genéreuse sur la période {i}"
        ].empty:
            dataset_cluster_explanations[
                f"Facteur d'abonnement pour {promo} sur la période {i}"
            ] = (
                dataset_cluster_explanations[
                    f"Facteur d'abonnement pour {promo} sur la période {i}"
                ]
                / dataset_cluster_explanations[
                    f"Facteur d'abonnement pour Semaine genéreuse sur la période {i}"
                ]
            )
        else:
            dataset_cluster_explanations[
                f"Facteur d'abonnement pour {promo} sur la période {i}"
            ] = float("inf")

dataset_cluster_explanations.drop(columns=["Dico"], inplace=True)

dataset_cluster_explanations["Periods"] = dataset_cluster_explanations["Cluster"].map(
    calculate_abonnements_moyens(dataset, periods)
)
print(dataset_cluster_explanations)
for i in range(1, 7, 1):
    dataset_cluster_explanations[f"Taux d'abonnement sur la période {i}"] = (
        dataset_cluster_explanations["Periods"].apply(lambda x: x[f"Période {i}"])
    )
dataset_cluster_explanations.drop(columns=["Periods"], inplace=True)
dataset_cluster_explanations = dataset_cluster_explanations.fillna(0)
dataset_cluster_explanations = dataset_cluster_explanations.round(1)


# Enregistrer la figure dans un fichier
nom_fichier_temporaire = "clusters_explanation_shortened.csv"
dataset_cluster_explanations.to_csv(nom_fichier_temporaire, index=False)
# Enregistrer le fichier temporaire sur S3
with fs.open(BUCKET + nom_fichier_temporaire, mode="wb") as file_out:
    with open(nom_fichier_temporaire, "rb") as temp_file:
        file_out.write(temp_file.read())
# Supprimer le fichier temporaire
os.remove(nom_fichier_temporaire)

dataset_cluster_explanations_shortened = dataset_cluster_explanations.copy()
dataset_cluster_explanations_shortened

In [ ]:
# Boucler à travers chaque ligne du DataFrame
liste_promotions = list(dataset["TYPE_PROMON"].unique())

# Créer une nouvelle figure
fig = plt.figure(figsize=(20, 16))

# Boucler à travers chaque ligne du DataFrame
for i in range(1, 7, 1):
    for index, row in dataset_cluster_explanations.iterrows():
        # Créer une liste de valeurs des colonnes 'Abonnement par jour pour{promo}'
        valeurs_promotions = [
            row[f"Facteur d'abonnement pour {promo} sur la période {i}"]
            for promo in liste_promotions
        ]

        # Ajouter un sous-plot à la figure pour le graphe camembert
        ax = fig.add_subplot(4, 2, index + 1)
        ax.pie(
            valeurs_promotions,
            labels=liste_promotions,
            autopct="%1.1f%%",
            startangle=140,
        )
        ax.axis("equal")  # Assurer un aspect circulaire
        ax.set_title(f'Cluster {int(row["Cluster"])}')

    # Ajuster la disposition pour éviter les chevauchements
    fig.tight_layout()

    # Enregistrer la figure dans un fichier
    nom_fichier_temporaire = "graph_camembert_clusters.png"
    plt.savefig(nom_fichier_temporaire)

    # Enregistrer le fichier temporaire sur S3
    with fs.open(BUCKET + nom_fichier_temporaire, mode="wb") as file_out:
        with open(nom_fichier_temporaire, "rb") as temp_file:
            file_out.write(temp_file.read())

    # Supprimer le fichier temporaire
    os.remove(nom_fichier_temporaire)

    # Afficher la figure
    plt.show()

In [ ]:
liste_promotions = ["Semaine genéreuse", "ODD 15 jours TC", "ODD 30 jours TC"]

# Créer une nouvelle figure à l'extérieur de la boucle
# fig = plt.figure(figsize=(20, 16))

# Boucler à travers chaque ligne du DataFrame
for i in range(1, 7, 1):
    fig = plt.figure(figsize=(20, 16))
    for index, row in dataset_cluster_explanations_shortened.iterrows():
        # Créer une liste de valeurs des colonnes 'Facteur d'abonnement pour {promo} sur la période {i}'
        valeurs_promotions = [
            row[f"Facteur d'abonnement pour {promo} sur la période {i}"]
            for promo in liste_promotions
        ]
        print(valeurs_promotions)
        # Calculer l'index du sous-graphique dans une grille de 6x8
        subplot_index = index + 1

        # Ajouter un sous-plot à la figure pour l'histogramme
        ax = fig.add_subplot(2, 4, subplot_index)
        ax.hist(
            valeurs_promotions, bins=10, color="skyblue", edgecolor="black", alpha=0.7
        )
        ax.set_title(f'Cluster {int(row["Cluster"])} - Période {i}')
        ax.set_xlabel("Promotion")
        ax.set_ylabel("Facteur")

        # Définir les étiquettes sur l'axe des y avec les noms des promotions
        ax.set_xticks(range(len(liste_promotions)))
        ax.set_xticklabels(liste_promotions)

    # Ajuster la disposition pour éviter les chevauchements
    fig.tight_layout()

    # Enregistrer la figure dans un fichier
    nom_fichier_temporaire = f"histogrammes_clusters_Period_{i}.png"
    plt.savefig(nom_fichier_temporaire)

    # Enregistrer le fichier temporaire sur S3
    with fs.open(BUCKET + nom_fichier_temporaire, mode="wb") as file_out:
        with open(nom_fichier_temporaire, "rb") as temp_file:
            file_out.write(temp_file.read())

    # Supprimer le fichier temporaire
    os.remove(nom_fichier_temporaire)

    # Afficher la figure
    plt.show()

In [ ]:
liste_promotions = ["Semaine genéreuse", "ODD 15 jours TC", "ODD 30 jours TC"]
liste_clusters = list(range(8))

# Boucler à travers chaque cluster
for cluster in liste_clusters:
    # Créer une figure pour ce cluster
    fig, axs = plt.subplots(2, 3, figsize=(12, 8))

    # Boucler à travers chaque période
    for i, ax in enumerate(axs.flatten(), start=1):
        # Récupérer les valeurs des promotions pour ce cluster et cette période
        valeurs_promotions = [
            dataset_cluster_explanations_shortened.iloc[cluster][
                f"Facteur d'abonnement pour {promo} sur la période {i}"
            ]
            for promo in liste_promotions
        ]

        # Créer l'histogramme pour ce cluster et cette période
        ax.bar(
            liste_promotions,
            valeurs_promotions,
            color="skyblue",
            edgecolor="black",
            alpha=0.7,
        )
        ax.set_title(f"Période {i}, Cluster {cluster}")
        ax.set_xlabel("Promotion")
        ax.set_ylabel("Facteur")

        # Faire pivoter les étiquettes de l'axe x pour une meilleure lisibilité
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right")

    # Ajuster la disposition pour éviter les chevauchements
    plt.tight_layout()

    # Enregistrer la figure dans un fichier
    nom_fichier_temporaire = f"histogrammes_shortened_cluster_{cluster}.png"
    plt.savefig(nom_fichier_temporaire)

    # Enregistrer le fichier temporaire sur S3
    with fs.open(
        BUCKET + "Batons_shortened/" + nom_fichier_temporaire, mode="wb"
    ) as file_out:
        with open(nom_fichier_temporaire, "rb") as temp_file:
            file_out.write(temp_file.read())

    # Supprimer le fichier temporaire
    os.remove(nom_fichier_temporaire)

    # Afficher la figure
    plt.show()

In [ ]:
liste_promotions = list(dataset["TYPE_PROMON"].unique())
print(liste_promotions)
liste_clusters = list(range(8))

# Boucler à travers chaque cluster
for cluster in liste_clusters:
    # Créer une figure pour ce cluster
    fig, axs = plt.subplots(2, 3, figsize=(12, 8))

    # Boucler à travers chaque période
    for i, ax in enumerate(axs.flatten(), start=1):
        # Récupérer les valeurs des promotions pour ce cluster et cette période
        valeurs_promotions = [
            dataset_cluster_explanations.iloc[cluster][
                f"Facteur d'abonnement pour {promo} sur la période {i}"
            ]
            for promo in liste_promotions
        ]

        # Créer l'histogramme pour ce cluster et cette période
        ax.bar(
            liste_promotions,
            valeurs_promotions,
            color="skyblue",
            edgecolor="black",
            alpha=0.7,
        )
        ax.set_title(f"Période {i}, Cluster {cluster}")
        ax.set_xlabel("Promotion")
        ax.set_ylabel("Facteur")

        # Faire pivoter les étiquettes de l'axe x pour une meilleure lisibilité
        plt.setp(ax.get_xticklabels(), rotation=45, ha="right")

    # Ajuster la disposition pour éviter les chevauchements
    plt.tight_layout()

    # Enregistrer la figure dans un fichier
    nom_fichier_temporaire = f"histogrammes_full_cluster_{cluster}.png"
    plt.savefig(nom_fichier_temporaire)

    # Enregistrer le fichier temporaire sur S3
    with fs.open(
        BUCKET + "Batons_full/" + nom_fichier_temporaire, mode="wb"
    ) as file_out:
        with open(nom_fichier_temporaire, "rb") as temp_file:
            file_out.write(temp_file.read())

    # Supprimer le fichier temporaire
    os.remove(nom_fichier_temporaire)

    # Afficher la figure
    plt.show()

In [ ]:
"""Trace les courbes de taux d'abonnement pour chaque cluster."""

for cluster in range(0, 8, 1):
    X = range(1, 7, 1)

    # Sélectionner la ligne correspondant au cluster 'Cluster1'
    cluster_row = dataset_cluster_explanations.loc[
        dataset_cluster_explanations["Cluster"] == cluster
    ]
    # Extraire les valeurs des 6 dernières colonnes
    Y = (cluster_row.iloc[:, -6:].values.tolist())[0]
    print(Y)

    plt.plot(X, Y, label=f"Cluster {cluster}")

# Ajouter un titre global et un label pour l'axe x
plt.xlabel("Période")
plt.suptitle("Taux d'abonnement par période pour chaque cluster")
plt.legend(loc="upper right")

# Enregistrer la figure dans un fichier
nom_fichier_temporaire = "graph_taux_abonnement.png"
plt.savefig(nom_fichier_temporaire)

# Enregistrer le fichier temporaire sur S3
with fs.open(BUCKET + nom_fichier_temporaire, mode="wb") as file_out:
    with open(nom_fichier_temporaire, "rb") as temp_file:
        file_out.write(temp_file.read())

# Supprimer le fichier temporaire
os.remove(nom_fichier_temporaire)

# Afficher le graphe
plt.show()

In [ ]:
dataset["DATE_PRISE_EFFET"].min(), dataset["DATE_PRISE_EFFET"].max()

In [ ]:
"""Trace les courbes de taux d'abonnement pour chaque cluster."""

for cluster in range(0, 8, 1):
    Y = {}
    for promo in ["Semaine genéreuse", "ODD 15 jours TC", "ODD 30 jours TC"]:
        X = list(range(1, 7, 1))
        list_columns = [
            f"Facteur d'abonnement pour {promo} sur la période {i}" for i in X
        ]
        Y[promo] = [
            dataset_cluster_explanations.iloc[
                cluster, dataset_cluster_explanations.columns.get_loc(column)
            ]
            for column in list_columns
        ]
        plt.plot(X, Y[promo], label=promo)

    # Ajouter un titre global et un label pour l'axe x
    plt.xlabel("Période")
    plt.suptitle(f"Facteur par période pour le cluster {cluster}")
    plt.legend(loc="upper right")

    # Enregistrer la figure dans un fichier
    nom_fichier_temporaire = (
        f"Graph_facteurs à travers le temps pour le cluster {cluster}.png"
    )
    plt.savefig(nom_fichier_temporaire)

    # Enregistrer le fichier temporaire sur S3
    with fs.open(BUCKET + nom_fichier_temporaire, mode="wb") as file_out:
        with open(nom_fichier_temporaire, "rb") as temp_file:
            file_out.write(temp_file.read())

    # Supprimer le fichier temporaire
    os.remove(nom_fichier_temporaire)

    # Afficher le graphe
    plt.show()

In [ ]:
df = dataset.copy()

# Convertir la colonne 'DATE_PRISE_EFFET' en type datetime
df["DATE_PRISE_EFFET"] = pd.to_datetime(df["DATE_PRISE_EFFET"])

# Liste des promotions à considérer
liste_promotions = ["Semaine genéreuse", "ODD 15 jours TC", "ODD 30 jours TC"]

# Initialiser une liste pour stocker les périodes par type de promotion
periodes_par_type = {}

# Parcourir chaque type de promotion dans liste_promotions
for type_promon in liste_promotions:
    # Filtrer le DataFrame pour inclure uniquement les lignes correspondant au type de promotion actuel
    df_type_promon = df[df["TYPE_PROMON"] == type_promon]

    # Calculer le nombre d'abonnements par jour
    abonnements_par_jour = df_type_promon.groupby("DATE_PRISE_EFFET").size()
    print(type(abonnements_par_jour))

    # Calculer la moyenne et l'écart-type des abonnements par jour
    moyenne = abonnements_par_jour.mean()
    ecart_type = abonnements_par_jour.std()

    # Déterminer le seuil pour les abonnements significatifs (95% des valeurs les plus élevées)
    # seuil = abonnements_par_jour.quantile(0.05)
    # seuil = moyenne - 1.96 * ecart_type
    seuil = (1 / 3) * moyenne

    # Identifier les jours où le nombre d'abonnements est significatif
    jours_significatifs = abonnements_par_jour[
        abonnements_par_jour >= seuil
    ].index.tolist()

    # Identifier les périodes consécutives de jours significatifs
    periodes = []
    periode = []
    for jour in jours_significatifs:
        if not periode or (jour - periode[-1]).days <= 4:
            periode.append(jour)
        else:
            periodes.append(periode)
            periode = [jour]
    if periode:
        periodes.append(periode)

    to_delete = [
        i for i in range(len(periodes)) if (periodes[i][-1] - periodes[i][0]).days <= 2
    ]
    c = 0
    for i in to_delete:
        del periodes[i - c]
        c += 1

    periodes_par_type[type_promon] = periodes

# Afficher les résultats
dico_periodes = {}
for type_promon, periodes in periodes_par_type.items():
    print(f"Type de promotion : {type_promon}")
    print("Périodes significatives :")
    dico_periodes[type_promon] = []
    for periode in periodes:
        print(
            "-", periode[0].strftime("%Y-%m-%d"), "à", periode[-1].strftime("%Y-%m-%d")
        )
        dico_periodes[type_promon] += [
            [periode[0].strftime("%Y-%m-%d"), periode[-1].strftime("%Y-%m-%d")]
        ]
    print(dico_periodes)


print(dico_periodes)

In [ ]:
# Votre dictionnaire
mon_dictionnaire = dico_periodes.copy()
if "ODD 30 jours TC" in mon_dictionnaire:
    del mon_dictionnaire["ODD 30 jours TC"]

print(mon_dictionnaire)

# Enregistrer la figure dans un fichier
nom_fichier_temporaire = "read_me.txt"
# Enregistrer le fichier temporaire sur S3
with open(nom_fichier_temporaire, "w") as fichier:
    # Écrire le dictionnaire dans le fichier au format JSON
    json.dump(mon_dictionnaire, fichier)

    # Ajouter des informations supplémentaires
    fichier.write("\n")
    fichier.write("Nous avons calculé des Périodes d'utilisation massives des ODD.\n")
    fichier.write(
        "Pour ODD15JTC et Semaine Généreuse, nous avons calculé des périodes sur lesquelles l'offre est massivement utilisée.\n"
    )
    fichier.write("\n")
    fichier.write("Les Périodes sont calculées de la sorte :\n")
    fichier.write(
        "    On considère comme période un ensemble de jours consécutifs pendant lesquels l'offre est significative au minimum tous les 4 jours.\n"
    )
    fichier.write(
        "    L'offre est significative au jour j si le nombre d'abonnements faits le jour J est supérieur à 1/3 de la moyenne. C'est une façon de se préserver\n"
    )
    fichier.write("    des faibles utilisations qui relèvent de très petites cibles.\n")
    fichier.write(
        "    En d'autres termes, sur une période, l'offre est utilisée en continue, mais il peut y avoir des creux de faible utilisation d'une durée maximale de 4 jours.\n"
    )
    fichier.write("    Si un creux dépasse les 4 jours, alors la période s'arrête.\n")

    fichier.write("\n")
    fichier.write(
        "Les Périodes sont ensuite utilisées de la façon suivante pour calculer notre indicateur:\n"
    )
    fichier.write(
        "    Prenons une période P d'utilisation d'ODD 15J TC. Notons P* et P** les périodes d'utilisation de Semaine Généreuse, respectivement\n"
    )
    fichier.write(
        "    la période la plus proche de P qui la précède et la plus proche de P qui la succède.\n"
    )
    fichier.write(
        "    Ensuite, on compare le nombre d'abonnement moyen sur P au nombre d'abonnement moyen sur P* et P**.\n"
    )
    fichier.write(
        "    En d'autres termes, on compare le nombre d'abonnements sur une période de promotions P au nombre d'abonnements sur les périodes de simple\n"
    )
    fichier.write("    Semaine Généreuse qui entourent P.\n")
    fichier.write("\n")
    fichier.write(
        "On obtient ainsi un indicateur appelé Facteur ODD-15JTC Periode i qui représente, dit d'une façon plus qualitative :\n"
    )
    fichier.write(
        "    Le facteur de réabonnement induit par l'usage d'une période d'ODD 15J TC.\n"
    )

# Enregistrer le fichier temporaire sur S3
with fs.open(
    BUCKET + "Periodes revues/" + nom_fichier_temporaire, mode="wb"
) as file_out:
    with open(nom_fichier_temporaire, "rb") as fichier:
        # Copier le contenu du fichier local vers le fichier sur S3
        file_out.write(fichier.read())

# Supprimer le fichier temporaire local
os.remove(nom_fichier_temporaire)

In [ ]:
from datetime import datetime, timedelta

# Exemple de dictionnaire de périodes
dico_periodes = {
    "promo1": [
        [datetime(2024, 1, 1), datetime(2024, 1, 5)],
        [datetime(2024, 1, 6), datetime(2024, 1, 10)],
    ],
    "promo2": [
        [datetime(2024, 2, 1), datetime(2024, 2, 5)],
        [datetime(2024, 2, 6), datetime(2024, 2, 10)],
    ],
}

# Transformer les périodes selon les règles spécifiées
type_promon = "promo1"
for i in range(len(dico_periodes[type_promon]) - 1):
    debut_suivant = dico_periodes[type_promon][i + 1][0]
    fin_actuelle = dico_periodes[type_promon][i][1]
    difference = (debut_suivant - fin_actuelle) / 2
    jour_milieu = fin_actuelle + difference
    dico_periodes[type_promon][i][1] = jour_milieu

# Afficher les périodes mises à jour
print(dico_periodes)

In [ ]:
liste_promotions = ["Semaine genéreuse", "ODD 15 jours TC", "ODD 30 jours TC"]


def fonction_ODD_15(
    group, dic_periods, cluster, type_promo, nombre_reabo_par_jour_dispo
):
    cluster_abonnements_moyens = {}
    for i, (debut_periode, fin_periode) in enumerate(
        dic_periods["ODD 15 jours TC"], start=1
    ):
        periode_data = group[
            (group["DATE_PRISE_EFFET"] >= debut_periode)
            & (group["DATE_PRISE_EFFET"] < fin_periode)
        ]
        abonnements_periode = len(periode_data)
        dates_diff = periode_data["DATE_PRISE_EFFET"].nunique()
        if dates_diff > 0:
            abonnements_moyens_periode = abonnements_periode / dates_diff
        else:
            abonnements_moyens_periode = 0
        cluster_abonnements_moyens[f"Période {i}"] = abonnements_moyens_periode
    if cluster not in nombre_reabo_par_jour_dispo:
        nombre_reabo_par_jour_dispo[cluster] = {}
    nombre_reabo_par_jour_dispo[cluster][type_promo] = cluster_abonnements_moyens
    return nombre_reabo_par_jour_dispo


def fonction_SG(group, dic_periods, cluster, type_promo, nombre_reabo_par_jour_dispo):
    # Parcourir chaque période de 'ODD 15 jours TC'
    liste_SG = []
    for periode_ODD in dic_periods["ODD 15 jours TC"]:
        # Initialiser les périodes précédente et suivante avec None
        periode_precedente = None
        periode_suivante = None
        print("dicoSG = ", dic_periods["Semaine genéreuse"])

        # Parcourir chaque période de 'Semaine genéreuse'
        for periode_SG in dic_periods["Semaine genéreuse"]:
            # Si la période de 'Semaine genéreuse' est avant la période de 'ODD 15 jours TC'
            print(
                "periode_SG[-1] <= periode_ODD[0] and periode_SG[-1] <= periode_ODD[1] ? Voyons : ",
                periode_SG[-1] <= periode_ODD[0],
                periode_SG[-1] <= periode_ODD[1],
            )
            if periode_SG[-1] < periode_ODD[-1] and periode_SG[0] < periode_ODD[0]:
                periode_precedente = periode_SG
            # Si la période de 'Semaine genéreuse' est après la période de 'ODD 15 jours TC'
            elif periode_SG[0] > periode_ODD[0] and periode_SG[-1] > periode_ODD[-1]:
                periode_suivante = periode_SG
                break  # On peut arrêter la recherche car les périodes sont triées par ordre croissant

        # Ajouter les résultats à liste_SG
        liste_SG.append([periode_precedente, periode_suivante])
    print("liste_SG = ", liste_SG)
    # Afficher les résultats
    for i, (periode_ODD, (periode_precedente, periode_suivante)) in enumerate(
        zip(dic_periods["ODD 15 jours TC"], liste_SG)
    ):
        print(f"Période ODD {i+1}: {periode_ODD}")
        print(f"Période précédente SG: {periode_precedente}")
        print(f"Période suivante SG: {periode_suivante}")
        print()

    cluster_abonnements_moyens = {}

    for i, list_sg in enumerate(liste_SG, start=1):
        print(type(None))
        print(list_sg[1])
        print([None, None])
        if list_sg[0] == None:
            debut_periode1 = None
            fin_periode1 = None
            debut_periode2 = list_sg[1][0]
            fin_periode2 = list_sg[1][1]
        elif list_sg[1] == None:
            debut_periode1 = list_sg[0][0]
            fin_periode1 = list_sg[0][1]
            debut_periode2 = None
            fin_periode2 = None
        else:
            debut_periode1 = list_sg[0][0]
            fin_periode1 = list_sg[0][1]
            debut_periode2 = list_sg[1][0]
            fin_periode2 = list_sg[1][1]

        if fin_periode1 == None:
            periode_data = group[
                (group["DATE_PRISE_EFFET"] >= debut_periode2)
                & (group["DATE_PRISE_EFFET"] < fin_periode2)
            ]
        elif debut_periode2 == None:
            periode_data = group[
                (group["DATE_PRISE_EFFET"] >= debut_periode1)
                & (group["DATE_PRISE_EFFET"] < fin_periode1)
            ]
        else:
            periode_data = group[
                (
                    (group["DATE_PRISE_EFFET"] >= debut_periode1)
                    & (group["DATE_PRISE_EFFET"] < fin_periode1)
                )
                | (
                    (group["DATE_PRISE_EFFET"] >= debut_periode2)
                    & (group["DATE_PRISE_EFFET"] < fin_periode2)
                )
            ]

        abonnements_periode = len(periode_data)
        dates_diff = periode_data["DATE_PRISE_EFFET"].nunique()
        if dates_diff > 0:
            abonnements_moyens_periode = abonnements_periode / dates_diff
        else:
            abonnements_moyens_periode = 0
        cluster_abonnements_moyens[f"Période {i}"] = abonnements_moyens_periode
    if cluster not in nombre_reabo_par_jour_dispo:
        nombre_reabo_par_jour_dispo[cluster] = {}
    nombre_reabo_par_jour_dispo[cluster][type_promo] = cluster_abonnements_moyens
    return nombre_reabo_par_jour_dispo


liste_promotions = ["Semaine genéreuse", "ODD 15 jours TC", "ODD 30 jours TC"]


def find_mean_with_promo_periods_(df, dic_periods):
    """Returns aa dictionnary"""
    print(dic_periods)
    data = df.copy()
    data["DATE_PRISE_EFFET"] = pd.to_datetime(data["DATE_PRISE_EFFET"])

    # Grouper les données par CLUSTER_8, TYPE_PROMON et Période
    nombre_reabo_par_jour_dispo = {}
    for (cluster, type_promo), group in data.groupby(["Cluster_8", "TYPE_PROMON"]):
        if type_promo == "Semaine genéreuse":
            nombre_reabo_par_jour_dispo = fonction_SG(
                group, dic_periods, cluster, type_promo, nombre_reabo_par_jour_dispo
            )
        elif type_promo == "ODD 15 jours TC":
            nombre_reabo_par_jour_dispo = fonction_ODD_15(
                group, dic_periods, cluster, type_promo, nombre_reabo_par_jour_dispo
            )
        elif type_promo == "ODD 30 jours TC":
            continue
        else:
            continue

    # Afficher les résultats
    for cluster, resultats_periode in nombre_reabo_par_jour_dispo.items():
        print(f"Cluster {cluster}:")
        for type_promo, resultats_periode_type in resultats_periode.items():
            print(f"  - Type promotion {type_promo}:")
            for periode, moyenne in resultats_periode_type.items():
                print(
                    f"      Période {periode}: Nombre de réabonnements moyens par jour de disponibilité : {moyenne}"
                )
    return nombre_reabo_par_jour_dispo


# Appeler la fonction pour calculer les abonnements moyens
find_mean_with_promo_periods_(dataset, dico_periodes)

In [ ]:
def calculate_abonnements_moyens(datata, periods):
    """Calcule le nombre d'abonnements moyens par jour de disponibilité pour chaque cluster sur chaque période."""
    df = datata.copy()
    df["DATE_PRISE_EFFET"] = pd.to_datetime(df["DATE_PRISE_EFFET"])
    nombre_abonnements_moyens = {}
    for cluster, group in df.groupby("Cluster_8"):
        cluster_abonnements_moyens = {}
        for i, (debut_periode, fin_periode) in enumerate(periods, start=1):
            print("Cluster ", cluster, "période ", i, (debut_periode, fin_periode))
            periode_data = group[
                (group["DATE_PRISE_EFFET"] >= debut_periode)
                & (group["DATE_PRISE_EFFET"] < fin_periode)
            ]
            # abonnements_periode = periode_data.groupby('ID_ABONNE')['DATE_PRISE_EFFET'].nunique().count()
            abonnements_periode = periode_data.shape[0]
            print("abonnements_periode :", abonnements_periode)
            if i < 6:
                jours_disponibilite = 6 * periode_data["ID_ABONNE"].nunique()
            elif i == 6:
                jours_disponibilite = 4 * periode_data["ID_ABONNE"].nunique()

            print("jours_disponibilite : ", jours_disponibilite)
            if jours_disponibilite > 0:
                abonnements_moyens_periode = abonnements_periode / jours_disponibilite
            else:
                abonnements_moyens_periode = 0
            cluster_abonnements_moyens[f"Période {i}"] = abonnements_moyens_periode
        nombre_abonnements_moyens[cluster] = cluster_abonnements_moyens
    return nombre_abonnements_moyens

In [ ]:
dataset_cluster_explanations = pd.DataFrame({"Cluster": list(range(8))})
print(dataset_cluster_explanations)

dataset_cluster_explanations["Pourcentage de la population"] = (
    dataset_cluster_explanations["Cluster"].map(find_percent(dataset))
)
dataset_cluster_explanations["Dico"] = dataset_cluster_explanations["Cluster"].map(
    find_mean_with_promo_periods_(dataset, dico_periodes)
)
print(dataset["TYPE_PROMON"].unique())
list_promo = ["Semaine genéreuse", "ODD 15 jours TC"]  # , 'ODD 30 jours TC']
for i in range(1, 11, 1):
    for promo in list_promo:
        dataset_cluster_explanations[
            f"Facteur d'abonnement pour {promo} sur la période {i}"
        ] = dataset_cluster_explanations["Dico"].apply(
            lambda x: x[promo][f"Période {i}"]
        )
        print(
            dataset_cluster_explanations[
                f"Facteur d'abonnement pour {promo} sur la période {i}"
            ]
        )


list_promo = ["ODD 15 jours TC"]
for promo in list_promo:
    for i in range(1, 11, 1):
        if not dataset_cluster_explanations[
            f"Facteur d'abonnement pour Semaine genéreuse sur la période {i}"
        ].empty:
            dataset_cluster_explanations[
                f"Facteur d'abonnement pour {promo} sur la période {i}"
            ] = (
                dataset_cluster_explanations[
                    f"Facteur d'abonnement pour {promo} sur la période {i}"
                ]
                / dataset_cluster_explanations[
                    f"Facteur d'abonnement pour Semaine genéreuse sur la période {i}"
                ]
            )
        else:
            dataset_cluster_explanations[
                f"Facteur d'abonnement pour {promo} sur la période {i}"
            ] = float("inf")

for i in range(1, 11, 1):
    dataset_cluster_explanations.drop(
        columns=[f"Facteur d'abonnement pour Semaine genéreuse sur la période {i}"],
        inplace=True,
    )
    dataset_cluster_explanations.rename(
        columns={
            f"Facteur d'abonnement pour ODD 15 jours TC sur la période {i}": f"Facteur ODD-15JTC Periode {i}"
        },
        inplace=True,
    )

dataset_cluster_explanations.drop(columns=["Dico"], inplace=True)

"""
dataset_cluster_explanations["Periods"] = dataset_cluster_explanations['Cluster'].map(calculate_abonnements_moyens(dataset, periods))
print(dataset_cluster_explanations)
for i in range(1, 7, 1):
    dataset_cluster_explanations[f"Taux d'abonnement sur la période {i}"] = dataset_cluster_explanations["Periods"].apply(lambda x: x[f"Période {i}"])
dataset_cluster_explanations.drop(columns = ['Periods'], inplace = True)
"""
dataset_cluster_explanations = dataset_cluster_explanations.fillna(0)
dataset_cluster_explanations = dataset_cluster_explanations.round(1)


# Enregistrer la figure dans un fichier
nom_fichier_temporaire = "clusters_explanation_ODD15JTC.csv"
dataset_cluster_explanations.to_csv(nom_fichier_temporaire, index=False)
# Enregistrer le fichier temporaire sur S3
with fs.open(
    BUCKET + "Periodes revues/" + nom_fichier_temporaire, mode="wb"
) as file_out:
    with open(nom_fichier_temporaire, "rb") as temp_file:
        file_out.write(temp_file.read())
# Supprimer le fichier temporaire
os.remove(nom_fichier_temporaire)

dataset_cluster_explanations_shortened = dataset_cluster_explanations.copy()
dataset_cluster_explanations_shortened

In [ ]:
"""Trace les courbes de taux d'abonnement pour chaque cluster."""

for cluster in range(0, 8, 1):
    X = list(range(1, 11, 1))
    list_columns = [f"Facteur ODD-15JTC Periode {i}" for i in X]
    Y = [
        dataset_cluster_explanations_shortened.iloc[
            cluster, dataset_cluster_explanations_shortened.columns.get_loc(column)
        ]
        for column in list_columns
    ]
    plt.plot(X, Y, label=cluster)

# Ajouter un titre global et un label pour l'axe x
plt.xlabel("Période")
plt.suptitle("Facteur ODD-15JTC")
plt.legend(loc="upper right")

# Enregistrer la figure dans un fichier
nom_fichier_temporaire = "Facteur ODD-15JTC.png"
plt.savefig(nom_fichier_temporaire)

# Enregistrer le fichier temporaire sur S3
with fs.open(
    BUCKET + "Periodes revues/" + nom_fichier_temporaire, mode="wb"
) as file_out:
    with open(nom_fichier_temporaire, "rb") as temp_file:
        file_out.write(temp_file.read())

# Supprimer le fichier temporaire
os.remove(nom_fichier_temporaire)

# Afficher le graphe
plt.show()